## Grocery calculator

In [1]:
import pandas as pd
import requests
import json
import regex as re
import numpy as np
import os
from dotenv import load_dotenv

### Setup API

In [2]:
load_dotenv()

True

In [3]:
url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
headers = {'Content-type': 'application/json'}
api_key = os.getenv('PROJECT_API_KEY')


In [4]:
print(f"API Key loaded: {api_key is not None}")

API Key loaded: True


### Load series data

In [5]:
df = pd.read_csv('data/series-data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   series_id    25 non-null     object
 1   item_name    25 non-null     object
 2   description  25 non-null     object
 3   unit         25 non-null     object
 4   category     25 non-null     object
 5   url          25 non-null     object
dtypes: object(6)
memory usage: 1.3+ KB


In [6]:
series_id = df['series_id']

In [7]:
data = {
    "seriesid": series_id.tolist(), 
    "startyear": "2024",
    "endyear": "2025",
    "registrationkey": api_key
}

In [8]:
response = requests.post(url, data=json.dumps(data), headers=headers)
response_data = response.json()

In [9]:
response_data

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 230,
 'message': [],
 'Results': {'series': [{'seriesID': 'APU0000701312',
    'data': [{'year': '2025',
      'period': 'M12',
      'periodName': 'December',
      'latest': 'true',
      'value': '1.076',
      'footnotes': [{}]},
     {'year': '2025',
      'period': 'M11',
      'periodName': 'November',
      'value': '1.065',
      'footnotes': [{}]},
     {'year': '2025',
      'period': 'M10',
      'periodName': 'October',
      'value': '-',
      'footnotes': [{'code': 'X',
        'text': 'Data unavailable due to the 2025 lapse in appropriations'}]},
     {'year': '2025',
      'period': 'M09',
      'periodName': 'September',
      'value': '1.060',
      'footnotes': [{}]},
     {'year': '2025',
      'period': 'M08',
      'periodName': 'August',
      'value': '1.059',
      'footnotes': [{}]},
     {'year': '2025',
      'period': 'M07',
      'periodName': 'July',
      'value': '1.068',
      'footnotes': [{}]},
     

In [10]:
response_df = pd.DataFrame(response_data['Results']['series'])

In [11]:
exploded_df = pd.json_normalize(response_data['Results']['series'], record_path='data')

In [12]:
exploded_df = response_df.explode('data')
exploded_df = pd.concat([exploded_df.drop(columns=['data']), exploded_df['data'].apply(pd.Series)], axis=1)
exploded_df

,seriesID,year,period,periodName,latest,value,footnotes
0,APU0000701312,2025,M12,December,true,1.076,[{}]
0,APU0000701312,2025,M11,November,NaN,1.065,[{}]
0,APU0000701312,2025,M10,October,NaN,-,"[{'code': 'X', 'text': 'Data unavailable due t..."
0,APU0000701312,2025,M09,September,NaN,1.060,[{}]
0,APU0000701312,2025,M08,August,NaN,1.059,[{}]
...,...,...,...,...,...,...,...
24,APU0000FF1101,2024,M05,May,NaN,4.118,[{}]
24,APU0000FF1101,2024,M04,April,NaN,4.061,[{}]
24,APU0000FF1101,2024,M03,March,NaN,4.106,[{}]
24,APU0000FF1101,2024,M02,February,NaN,4.105,[{}]


In [13]:
exploded_df.to_csv('bls_data_nov_2025.csv', index=False)

In [14]:
df2 = exploded_df.merge(df[['series_id', 'item_name']], left_on='seriesID', right_on='series_id', how='left')


In [15]:
df2.drop(columns=['series_id'], inplace=True)

In [16]:
df2

,seriesID,year,period,periodName,latest,value,footnotes,item_name
0,APU0000701312,2025,M12,December,true,1.076,[{}],Rice
1,APU0000701312,2025,M11,November,NaN,1.065,[{}],Rice
2,APU0000701312,2025,M10,October,NaN,-,"[{'code': 'X', 'text': 'Data unavailable due t...",Rice
3,APU0000701312,2025,M09,September,NaN,1.060,[{}],Rice
4,APU0000701312,2025,M08,August,NaN,1.059,[{}],Rice
...,...,...,...,...,...,...,...,...
592,APU0000FF1101,2024,M05,May,NaN,4.118,[{}],Chicken breast
593,APU0000FF1101,2024,M04,April,NaN,4.061,[{}],Chicken breast
594,APU0000FF1101,2024,M03,March,NaN,4.106,[{}],Chicken breast
595,APU0000FF1101,2024,M02,February,NaN,4.105,[{}],Chicken breast


In [17]:
df2.to_csv('data/raw-data.csv', index=False)

In [18]:
df2['date'] = df2['periodName'] + ' ' + df2['year']

In [19]:
df2

,seriesID,year,period,periodName,latest,value,footnotes,item_name,date
0,APU0000701312,2025,M12,December,true,1.076,[{}],Rice,December 2025
1,APU0000701312,2025,M11,November,NaN,1.065,[{}],Rice,November 2025
2,APU0000701312,2025,M10,October,NaN,-,"[{'code': 'X', 'text': 'Data unavailable due t...",Rice,October 2025
3,APU0000701312,2025,M09,September,NaN,1.060,[{}],Rice,September 2025
4,APU0000701312,2025,M08,August,NaN,1.059,[{}],Rice,August 2025
...,...,...,...,...,...,...,...,...,...
592,APU0000FF1101,2024,M05,May,NaN,4.118,[{}],Chicken breast,May 2024
593,APU0000FF1101,2024,M04,April,NaN,4.061,[{}],Chicken breast,April 2024
594,APU0000FF1101,2024,M03,March,NaN,4.106,[{}],Chicken breast,March 2024
595,APU0000FF1101,2024,M02,February,NaN,4.105,[{}],Chicken breast,February 2024


In [20]:
df2['value'] = pd.to_numeric(df2['value'], errors='coerce').round(2)

In [21]:
df2['value'] = df2['value'].astype(float).round(2)
df2

,seriesID,year,period,periodName,latest,value,footnotes,item_name,date
0,APU0000701312,2025,M12,December,true,1.08,[{}],Rice,December 2025
1,APU0000701312,2025,M11,November,NaN,1.06,[{}],Rice,November 2025
2,APU0000701312,2025,M10,October,NaN,NaN,"[{'code': 'X', 'text': 'Data unavailable due t...",Rice,October 2025
3,APU0000701312,2025,M09,September,NaN,1.06,[{}],Rice,September 2025
4,APU0000701312,2025,M08,August,NaN,1.06,[{}],Rice,August 2025
...,...,...,...,...,...,...,...,...,...
592,APU0000FF1101,2024,M05,May,NaN,4.12,[{}],Chicken breast,May 2024
593,APU0000FF1101,2024,M04,April,NaN,4.06,[{}],Chicken breast,April 2024
594,APU0000FF1101,2024,M03,March,NaN,4.11,[{}],Chicken breast,March 2024
595,APU0000FF1101,2024,M02,February,NaN,4.11,[{}],Chicken breast,February 2024


In [22]:
sorted_columns = sorted(df2['date'].unique(), key=lambda x: pd.to_datetime(x))
pivot_sorted = df2.pivot(index='seriesID', columns='date', values='value')[sorted_columns]
pivot_sorted

date,January 2024,February 2024,March 2024,April 2024,May 2024,June 2024,July 2024,August 2024,September 2024,October 2024,...,March 2025,April 2025,May 2025,June 2025,July 2025,August 2025,September 2025,October 2025,November 2025,December 2025
seriesID,,,,,,,,,,,,,,,,,,,,,
APU0000701312,1.00,1.00,1.01,1.02,1.00,1.02,1.04,1.08,1.08,1.06,...,1.04,1.03,1.07,1.06,1.07,1.06,1.06,NaN,1.06,1.08
APU0000701322,1.42,1.43,1.42,1.42,1.38,1.44,1.43,1.42,1.43,1.44,...,1.38,1.36,1.32,1.32,1.31,1.29,1.20,NaN,1.30,1.31
APU0000702111,2.03,2.01,2.00,2.00,1.97,1.97,1.98,1.95,1.98,1.94,...,1.88,1.91,1.88,1.86,1.85,1.84,1.87,NaN,1.79,1.83
APU0000702421,5.02,5.25,5.14,5.12,5.12,5.02,5.10,5.05,5.17,4.85,...,4.86,4.96,5.05,5.11,5.26,5.12,5.15,NaN,5.33,5.13
APU0000703113,6.78,6.65,6.73,6.81,6.85,6.89,7.17,6.92,7.22,7.22,...,7.48,7.55,7.69,7.67,8.04,7.95,8.18,NaN,8.23,8.25
APU0000703511,8.11,7.88,8.03,8.11,8.25,7.93,8.06,8.23,8.15,8.15,...,8.55,8.63,8.40,8.46,8.69,9.08,9.36,NaN,9.66,9.88
APU0000704111,6.61,6.56,6.61,6.64,6.82,6.83,6.88,6.78,6.96,6.87,...,6.98,7.01,6.98,7.10,7.12,7.21,7.29,NaN,6.93,6.76
APU0000704312,5.53,5.67,5.57,5.59,5.67,5.57,5.73,5.63,5.63,5.67,...,5.60,5.47,5.45,5.38,5.68,5.43,5.48,NaN,5.46,5.36
APU0000708111,2.52,3.00,2.99,2.86,2.70,2.72,3.08,3.20,3.82,3.37,...,6.23,5.12,4.55,3.78,3.60,3.59,3.49,NaN,2.86,2.71


In [23]:
df3 = pivot_sorted.merge(df, left_on='seriesID', right_on='series_id', how='left')

In [24]:
column_order = ['series_id', 'item_name', 'description', 'unit', 'category', 'url'] + [col for col in df3.columns if col not in ['series_id', 'item_name', 'description', 'unit', 'category', 'url']]
df4 = df3[column_order]
df4

,series_id,item_name,description,unit,category,url,January 2024,February 2024,March 2024,April 2024,...,March 2025,April 2025,May 2025,June 2025,July 2025,August 2025,September 2025,October 2025,November 2025,December 2025
0,APU0000701312,Rice,"Rice, white, long grain, uncooked, per lb.",per lb.,Pantry,https://fred.stlouisfed.org/graph/fredgraph.cs...,1.00,1.00,1.01,1.02,...,1.04,1.03,1.07,1.06,1.07,1.06,1.06,NaN,1.06,1.08
1,APU0000701322,Pasta,"Spaghetti and macaroni, per lb.",per lb.,Pantry,https://fred.stlouisfed.org/graph/fredgraph.cs...,1.42,1.43,1.42,1.42,...,1.38,1.36,1.32,1.32,1.31,1.29,1.20,NaN,1.30,1.31
2,APU0000702111,White bread,"Bread, white, pan, per lb.",per lb.,Pantry,https://fred.stlouisfed.org/graph/fredgraph.cs...,2.03,2.01,2.00,2.00,...,1.88,1.91,1.88,1.86,1.85,1.84,1.87,NaN,1.79,1.83
3,APU0000702421,Cookies,"Cookies, chocolate chip, per lb.",per lb.,Snacks,https://fred.stlouisfed.org/graph/fredgraph.cs...,5.02,5.25,5.14,5.12,...,4.86,4.96,5.05,5.11,5.26,5.12,5.15,NaN,5.33,5.13
4,APU0000703113,Ground beef,"Ground beef, lean and extra lean, per lb.",per lb.,Meat,https://fred.stlouisfed.org/graph/fredgraph.cs...,6.78,6.65,6.73,6.81,...,7.48,7.55,7.69,7.67,8.04,7.95,8.18,NaN,8.23,8.25
5,APU0000703511,Steak,"Steak, round, USDA Choice, boneless, per lb.",per lb.,Meat,https://fred.stlouisfed.org/graph/fredgraph.cs...,8.11,7.88,8.03,8.11,...,8.55,8.63,8.40,8.46,8.69,9.08,9.36,NaN,9.66,9.88
6,APU0000704111,Bacon,"Bacon, sliced, per lb.",per lb.,Meat,https://fred.stlouisfed.org/graph/fredgraph.cs...,6.61,6.56,6.61,6.64,...,6.98,7.01,6.98,7.10,7.12,7.21,7.29,NaN,6.93,6.76
7,APU0000704312,Ham,"Ham, boneless, excluding canned, per lb.",per lb.,Meat,https://fred.stlouisfed.org/graph/fredgraph.cs...,5.53,5.67,5.57,5.59,...,5.60,5.47,5.45,5.38,5.68,5.43,5.48,NaN,5.46,5.36
8,APU0000708111,Eggs,"Eggs, grade A, large, per doz.",per doz.,Dairy,https://fred.stlouisfed.org/graph/fredgraph.cs...,2.52,3.00,2.99,2.86,...,6.23,5.12,4.55,3.78,3.60,3.59,3.49,NaN,2.86,2.71
9,APU0000709112,Milk,"Milk, fresh, whole, fortified, per gal.",per gal.,Dairy,https://fred.stlouisfed.org/graph/fredgraph.cs...,3.96,3.94,3.89,3.87,...,4.05,4.07,4.02,4.03,4.16,4.17,4.13,NaN,4.00,4.05


In [25]:
df4

,series_id,item_name,description,unit,category,url,January 2024,February 2024,March 2024,April 2024,...,March 2025,April 2025,May 2025,June 2025,July 2025,August 2025,September 2025,October 2025,November 2025,December 2025
0,APU0000701312,Rice,"Rice, white, long grain, uncooked, per lb.",per lb.,Pantry,https://fred.stlouisfed.org/graph/fredgraph.cs...,1.00,1.00,1.01,1.02,...,1.04,1.03,1.07,1.06,1.07,1.06,1.06,NaN,1.06,1.08
1,APU0000701322,Pasta,"Spaghetti and macaroni, per lb.",per lb.,Pantry,https://fred.stlouisfed.org/graph/fredgraph.cs...,1.42,1.43,1.42,1.42,...,1.38,1.36,1.32,1.32,1.31,1.29,1.20,NaN,1.30,1.31
2,APU0000702111,White bread,"Bread, white, pan, per lb.",per lb.,Pantry,https://fred.stlouisfed.org/graph/fredgraph.cs...,2.03,2.01,2.00,2.00,...,1.88,1.91,1.88,1.86,1.85,1.84,1.87,NaN,1.79,1.83
3,APU0000702421,Cookies,"Cookies, chocolate chip, per lb.",per lb.,Snacks,https://fred.stlouisfed.org/graph/fredgraph.cs...,5.02,5.25,5.14,5.12,...,4.86,4.96,5.05,5.11,5.26,5.12,5.15,NaN,5.33,5.13
4,APU0000703113,Ground beef,"Ground beef, lean and extra lean, per lb.",per lb.,Meat,https://fred.stlouisfed.org/graph/fredgraph.cs...,6.78,6.65,6.73,6.81,...,7.48,7.55,7.69,7.67,8.04,7.95,8.18,NaN,8.23,8.25
5,APU0000703511,Steak,"Steak, round, USDA Choice, boneless, per lb.",per lb.,Meat,https://fred.stlouisfed.org/graph/fredgraph.cs...,8.11,7.88,8.03,8.11,...,8.55,8.63,8.40,8.46,8.69,9.08,9.36,NaN,9.66,9.88
6,APU0000704111,Bacon,"Bacon, sliced, per lb.",per lb.,Meat,https://fred.stlouisfed.org/graph/fredgraph.cs...,6.61,6.56,6.61,6.64,...,6.98,7.01,6.98,7.10,7.12,7.21,7.29,NaN,6.93,6.76
7,APU0000704312,Ham,"Ham, boneless, excluding canned, per lb.",per lb.,Meat,https://fred.stlouisfed.org/graph/fredgraph.cs...,5.53,5.67,5.57,5.59,...,5.60,5.47,5.45,5.38,5.68,5.43,5.48,NaN,5.46,5.36
8,APU0000708111,Eggs,"Eggs, grade A, large, per doz.",per doz.,Dairy,https://fred.stlouisfed.org/graph/fredgraph.cs...,2.52,3.00,2.99,2.86,...,6.23,5.12,4.55,3.78,3.60,3.59,3.49,NaN,2.86,2.71
9,APU0000709112,Milk,"Milk, fresh, whole, fortified, per gal.",per gal.,Dairy,https://fred.stlouisfed.org/graph/fredgraph.cs...,3.96,3.94,3.89,3.87,...,4.05,4.07,4.02,4.03,4.16,4.17,4.13,NaN,4.00,4.05


In [26]:
df4.to_csv('data/clean_data.csv', index=False)